In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
df = pd.read_csv("C:/Users/pkitt/Downloads/playground-series-s5e6/train.csv")

In [4]:
df.cat =df.select_dtypes(include=[object]) 

C:\Users\pkitt\AppData\Local\Temp\ipykernel_15964\1653857449.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.cat =df.select_dtypes(include=[object])


In [5]:
df.num=df.select_dtypes(include=[np.number])

C:\Users\pkitt\AppData\Local\Temp\ipykernel_15964\610118065.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.num=df.select_dtypes(include=[np.number])


In [6]:
df.cat.head()

,Soil Type,Crop Type,Fertilizer Name
0,Clayey,Sugarcane,28-28
1,Sandy,Millets,28-28
2,Sandy,Millets,17-17-17
3,Sandy,Barley,10-26-26
4,Red,Paddy,DAP


In [7]:
from sklearn.preprocessing import LabelEncoder

In [8]:
le = LabelEncoder()

In [9]:
df.cat["Soil Type"] = le.fit_transform(df.cat["Soil Type"])

In [10]:
df.cat["Crop Type"] = le.fit_transform(df.cat["Crop Type"])

In [11]:
df.cat["Fertilizer Name"] = le.fit_transform(df.cat["Fertilizer Name"])

In [12]:
df.cat.tail(20)

,Soil Type,Crop Type,Fertilizer Name
749980,0,10,5
749981,2,2,2
749982,3,8,0
749983,4,3,2
749984,4,6,1
749985,3,3,0
749986,0,10,3
749987,3,3,6
749988,0,3,1
749989,0,4,4


In [13]:
df_train = pd.concat([df.num,df.cat],axis=1)

In [14]:
X = df_train[["Temparature","Humidity","Moisture","Crop Type","Soil Type"]].values

In [15]:
y = df_train[["Fertilizer Name"]].values

In [16]:
import torch

x_tensor = torch.tensor(X, dtype=torch.float)  # X is your feature DataFrame
y_tensor = torch.tensor(y, dtype=torch.long).squeeze()  # y is your label DataFrame

In [17]:
y = y_tensor.cpu().numpy().ravel()

In [18]:
X = x_tensor.cpu().numpy()

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
from sklearn.ensemble import RandomForestClassifier

In [21]:
RFC=RandomForestClassifier(n_estimators=8)

In [22]:
fit=RFC.fit(X,y)

In [23]:
RFC.fit(X,y)

RandomForestClassifier(n_estimators=8)

In [24]:
fit.score(X,y)

0.6721853333333333

In [25]:
from sklearn.preprocessing import StandardScaler

In [27]:
scaler = StandardScaler()
X_np_scaled = scaler.fit_transform(X)
y_np = y_tensor.cpu().numpy().ravel()
RFC.fit(X_np_scaled, y.ravel())
RFC_probs = RFC.predict_proba(X_np_scaled)

unique_classes = np.unique(y_np)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# X and y already prepped
rf = RandomForestClassifier(n_estimators=150, class_weight='balanced', random_state=42)
rf.fit(X_np_scaled, y_np.ravel())

# Get class probabilities for each node (shape: [N, num_classes])
rf_probs = rf.predict_proba(X_np_scaled)

In [29]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
x_combined = np.concatenate([X_np_scaled, RFC_probs], axis=1)
x_tensor = torch.tensor(x_combined, dtype=torch.float).to(device)

In [30]:
import torch.nn as nn
class SimpleGCN(nn.Module):
    def __init__(self, in_channels, hidden_dim, num_classes):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.conv3 = GCNConv(hidden_dim, num_classes)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        return self.conv3(x, edge_index)

In [31]:
from torch_geometric.nn import GCNConv

In [32]:
import torch

x_tensor = torch.tensor(X, dtype=torch.float)  # X is your feature DataFrame
y_tensor = torch.tensor(y, dtype=torch.long).squeeze()  # y is your label DataFrame

In [33]:
model = SimpleGCN(in_channels=x_tensor.shape[1], hidden_dim=128, num_classes=len(torch.unique(y_tensor))).to(device)

In [34]:
x_fused = torch.tensor(X, dtype=torch.float)

In [35]:

from sklearn.neighbors import NearestNeighbors
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
X_np = x_tensor.cpu().numpy()
nbrs = NearestNeighbors(n_neighbors=8).fit(X_np)
_, indices = nbrs.kneighbors(X_np)

rows = np.repeat(np.arange(len(X_np)), indices.shape[1])
cols = indices.flatten()
edge_index_np = np.stack([rows, cols], axis=0)
edge_index = torch.tensor(edge_index_np, dtype=torch.long)
edge_index = torch.cat([edge_index, edge_index[[1, 0]]], dim=1)  # Make undirected
edge_index = edge_index.to(device)
assert edge_index.max().item() < x_tensor.shape[0]

In [36]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=5e-4)

In [37]:
import torch.nn.functional as F
import matplotlib.pyplot as plt

# Track performance
loss_history = []
acc_history = []

model.train()
for epoch in range(1, 51):  # Adjust number of epochs as needed
    optimizer.zero_grad()

    out = model(x_tensor, edge_index)  # Forward pass
    loss = F.cross_entropy(out, y_tensor)  # Loss computation
    loss.backward()
    optimizer.step()

    # Accuracy
    pred = out.argmax(dim=1)
    acc = (pred == y_tensor).float().mean().item()

    loss_history.append(loss.item())
    acc_history.append(acc)

    print(f"Epoch {epoch:02d} | Loss: {loss.item():.4f} | Accuracy: {acc:.4f}")

Epoch 01 | Loss: 5.6988 | Accuracy: 0.1511
Epoch 02 | Loss: 4.7917 | Accuracy: 0.1478
Epoch 03 | Loss: 3.8123 | Accuracy: 0.1354
Epoch 04 | Loss: 3.8438 | Accuracy: 0.1486
Epoch 05 | Loss: 3.6323 | Accuracy: 0.1488
Epoch 06 | Loss: 3.1255 | Accuracy: 0.1468
Epoch 07 | Loss: 2.7476 | Accuracy: 0.1465
Epoch 08 | Loss: 2.4459 | Accuracy: 0.1474
Epoch 09 | Loss: 2.2700 | Accuracy: 0.1419
Epoch 10 | Loss: 2.5044 | Accuracy: 0.1328
Epoch 11 | Loss: 2.6896 | Accuracy: 0.1332
Epoch 12 | Loss: 2.5743 | Accuracy: 0.1376
Epoch 13 | Loss: 2.3750 | Accuracy: 0.1427
Epoch 14 | Loss: 2.2459 | Accuracy: 0.1463
Epoch 15 | Loss: 2.2134 | Accuracy: 0.1475
Epoch 16 | Loss: 2.2003 | Accuracy: 0.1505
Epoch 17 | Loss: 2.1911 | Accuracy: 0.1510
Epoch 18 | Loss: 2.2156 | Accuracy: 0.1513
Epoch 19 | Loss: 2.2610 | Accuracy: 0.1507
Epoch 20 | Loss: 2.2160 | Accuracy: 0.1497
Epoch 21 | Loss: 2.0926 | Accuracy: 0.1509
Epoch 22 | Loss: 2.0058 | Accuracy: 0.1498
Epoch 23 | Loss: 2.0014 | Accuracy: 0.1433
Epoch 24 | 